# Introduction

* Script to plot volume data using mayavi (mlab)
* Examples of isosurface and volume rendering is available
* Close each mlab window in order for the script to start plotting the next figure

# Load libraries

In [1]:
# import libraries
import os
# for math
import numpy as np
# for netcdf file
# os.system("pip install netCDF4")
import netCDF4
from netCDF4 import Dataset

# plotting
from mayavi import mlab
from tvtk.util import ctf
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.colors import LinearSegmentedColormap

# Custom functions

In [2]:
# to reverse colormap of figure object 'obj'
def colormap_reverse(obj):
    lut = obj.module_manager.scalar_lut_manager.lut.table.to_array()
    ilut = lut[::-1]
    # putting LUT back in the surface object
    obj.module_manager.scalar_lut_manager.lut.table = ilut
    # forcing to update the figure once we have changed the LUT
    mlab.draw()

In [3]:
def compute_vorticity(u, v, w, dx, dy, dz):
    """
    Compute vorticity in 3D using velocity fields (u, v, w). Assuming uniform grid.

    Parameters:
        u (ndarray): Velocity component in the x-direction.
        v (ndarray): Velocity component in the y-direction.
        w (ndarray): Velocity component in the z-direction.
        dx (float): Grid spacing in the x-direction.
        dy (float): Grid spacing in the y-direction.
        dz (float): Grid spacing in the z-direction.

    Returns:
        vorticity_x (ndarray): Vorticity component in the x-direction.
        vorticity_y (ndarray): Vorticity component in the y-direction.
        vorticity_z (ndarray): Vorticity component in the z-direction.
    """
    # Calculate derivatives
    du_dx, du_dy, du_dz = np.gradient(u, dx, dy, dz)
    dv_dx, dv_dy, dv_dz = np.gradient(v, dx, dy, dz)
    dw_dx, dw_dy, dw_dz = np.gradient(w, dx, dy, dz)
    
    # Compute vorticity
    vorticity_x = dw_dy - dv_dz
    vorticity_y = du_dz - dw_dx
    vorticity_z = dv_dx - du_dy
    
    return vorticity_x, vorticity_y, vorticity_z

In [4]:
def compute_qcriterion(u, v, w, dx, dy, dz, norm=True):
    # Calculate derivatives
    du_dx, du_dy, du_dz = np.gradient(u, dx, dy, dz)
    dv_dx, dv_dy, dv_dz = np.gradient(v, dx, dy, dz)
    dw_dx, dw_dy, dw_dz = np.gradient(w, dx, dy, dz)

    # Compute vorticity
    vorticity_x = dw_dy - dv_dz
    vorticity_y = du_dz - dw_dx
    vorticity_z = dv_dx - du_dy
    
    Omega = 0.5 * (vorticity_z**2 + vorticity_y**2 + vorticity_x**2)
    S     = du_dx**2 + dv_dy**2 + dw_dz**2 + (0.5 * ( (dv_dx + du_dy)**2 + (dw_dx + du_dz)**2 + (dv_dz + dw_dy)**2) )
    Q = 0.5 * (Omega - S)
    
    if norm: # min-max norm to [-1,1]
        a = -1; b = 1
        Q = a + ( (Q - np.min(Q[:]))*(b - a) / (np.max(Q[:]) - np.min(Q[:])) )
    return Q

In [5]:
# custom blue-black-red continous colormap called `BlueBlackRed`
cdict = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 1.0, 1.0)),

         'green': ((0.0, 0.0, 0.0),
                   (1.0, 0.0, 0.0)),

         'blue':  ((0.0, 1.0, 1.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 0.0, 0.0))
        }
bbr = LinearSegmentedColormap('BlueBlackRed', cdict)
plt.register_cmap(cmap=bbr)
# cm.unregister_cmap('BlueBlackRed')

/var/folders/30/4m4szwf162x06l9rfddfgp90c0khl7/T/ipykernel_1718/3900880352.py:14: MatplotlibDeprecationWarning: The register_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps.register(name)`` instead.
  plt.register_cmap(cmap=bbr)


# User input

In [6]:
# file path
# HDdir = '/Users/81a/Desktop/Projects/Climate/miniWeatherML/experiments/supercell_example/jupyternotebooks'
# loadpath = f'{HDdir}/supercell_3d_weno9_acoust_acoust_advect_1000x1000x200.nc'
HDdir = './Data'
t_smpl = 5
loadpath = f'{HDdir}/test_{t_smpl:08d}.nc'

# Load data

In [7]:
# load NETCDF file
nc_data = Dataset(loadpath,'r')

# position and time vectors
x = np.array(nc_data.variables['x'][:])
y = np.array(nc_data.variables['y'][:])
z = np.array(nc_data.variables['z'][:])

# position meshgrid
X, Y, Z = np.meshgrid(x, y, z)

# read necessary variables
TKE = np.array(nc_data.variables['TKE'][:,:,:,0]).T
watervap = np.array(nc_data.variables['water_vapor'][:,:,:,0]).T
temp = np.array(nc_data.variables['temperature'][:,:,:,0]).T
u = np.array(nc_data.variables['uvel'][:,:,:,0]).T
v = np.array(nc_data.variables['vvel'][:,:,:,0]).T
w = np.array(nc_data.variables['wvel'][:,:,:,0]).T

In [8]:
dx = np.diff(x)[0]; dy = np.diff(y)[0]; dz = np.diff(z)[0]
Q = compute_qcriterion(u, v, w, dx, dy, dz, norm=True)

In [9]:
Umag = u**2 + v**2 + w**2

# Plotting

## Iso-surface plot

In [10]:
# vector = watervap
# scalar1 = cloud
# scalar_th1 = 0.0001
# scalar2 = precip
# scalar_th2 = 0.007
# # 0.0007 for cloud
# # 0.007 for precip
# bgcolor = (0,0,0)
# # deep skyblue(0, 0.7490196078431373, 1.0)
# # skyblue (0.5294117647058824, 0.807843137254902, 0.9215686274509803)

# fig = mlab.figure(fgcolor=(1, 1, 1), bgcolor=bgcolor, size=(1024, 768))

# # plot bottom slice
# obj = mlab.volume_slice(vector, plane_orientation='z_axes', line_width=0, slice_index=0, colormap='hot')
# lut = obj.module_manager.scalar_lut_manager.lut.table.to_array()
# obj.module_manager.scalar_lut_manager.lut.table = lut[0:int(len(lut)/2), :]
# mlab.draw()
# colormap_reverse(obj)

# # plot isosurface inner
# obj = mlab.contour3d(scalar2, contours=[scalar_th2], colormap='winter', opacity=1)
# # colormap_reverse(obj)


# # plot isosurface 2
# obj = mlab.contour3d(scalar1, contours=[scalar_th1], colormap='black-white', transparent=True, opacity=0.1)
# colormap_reverse(obj)


# # adjust camera
# mlab.view(azimuth=60, elevation=40, distance=1.8*len(x), focalpoint=[0.5*len(x), 0.6*len(y), 0.5*len(z)], roll=None, reset_roll=True, figure=fig)

# # # save fig
# # # filename = f'sample_plot_precip_Uvelocity_t{t_smpl}.png'
# # filename = f'sample_plot_cloud_Uvelocity_t{t_smpl}_multi-contour.png'
# # mlab.savefig(filename, size=(180,180), figure=None, magnification='auto')

# mlab.show()

## Volume rendering

### Sample

In [11]:
# bgcolor = (0,0,0) # background color
# fig = mlab.figure(fgcolor=(1, 1, 1), bgcolor=bgcolor, size=(3072, 2304))

# # plot bottom slice
# slc = 50  # slice number of the actual slice to be plotted
# # duplicate value of slice to all the other slices
# vector = w.copy()
# vector[:,:,:] = vector[:,:,slc].reshape( (len(x), len(y), 1) )
# # plot slice but the bottom (0th) slice. It'll have the value of slice `slc`
# mag = np.abs(vector).max()
# obj = mlab.volume_slice(vector, plane_orientation='z_axes', slice_index=0, line_width=0, colormap='bwr', vmin=-mag, vmax=mag)
# # matplotlib colormap
# values = np.linspace(0., 1., 256)
# cmap = cm.get_cmap('BlueBlackRed')(values.copy()) * 255
# cmap = cmap.astype('uint8')
# obj.module_manager.scalar_lut_manager.lut.table = cmap
# mlab.draw()

# # # precip
# # volume = mlab.pipeline.volume(mlab.pipeline.scalar_field(precip), vmin=0.008, vmax=0.009, color=(0,1,1))

# # # cloud
# # volume = mlab.pipeline.volume(mlab.pipeline.scalar_field(cloud), vmin=0.00007, vmax=0.002, color=(1,1,1))

# # Q-criterion
# volume = mlab.pipeline.volume(mlab.pipeline.scalar_field(Q), vmin=-0.21, vmax=-0.2, color=(0,1,1))
# # volume.actor.property.scalar_visibility = False  # Disable the scalar visibility for isosurface

# # # Define the scalar field for coloring
# # Umag = u**2 + v**2 + w**2
# # scalar_field = mlab.pipeline.scalar_field(Umag)
# # # Create an isosurface and color it by Umag
# # isosurface = mlab.pipeline.iso_surface(Q, color=(0,1,1))
# # isosurface.actor.mapper.scalar_visibility = False  # Disable the scalar visibility for isosurface
# # isosurface.actor.mapper.input_connection = scalar_field.output_port  # Connect Umag as the scalar field for coloring

# # adjust camera
# mlab.view(azimuth=60, elevation=70, distance=1.2*len(x), focalpoint=[0.5*len(x), 0.6*len(y), 0.5*len(z)], roll=None, reset_roll=True, figure=fig)

# # # save figure
# # filename = f'sample_volume_cloud_precip_t{t_smpl}.png'
# # mlab.savefig(filename, size=(50,50), figure=None, magnification='auto') # change size for adjusting resolution

# mlab.show()

### Single turbine

In [12]:
# bgcolor = (1,1,1) # background color
# fig = mlab.figure(fgcolor=(1, 1, 1), bgcolor=bgcolor, size=(3072, 2304))

# # plot bottom slice
# slc = 75  # slice number of the actual slice to be plotted
# # duplicate value of slice to all the other slices
# vector = w.copy()
# vector[:,:,:] = vector[:,:,slc].reshape( (len(x), len(y), 1) )
# # plot slice but the bottom (0th) slice. It'll have the value of slice `slc`
# mag = np.abs(vector).max()
# obj = mlab.volume_slice(vector, plane_orientation='z_axes', slice_index=0, line_width=0, colormap='bwr', vmin=-1, vmax=1)
# # matplotlib colormap
# values = np.linspace(0., 1., 256)
# cmap = cm.get_cmap('jet')(values.copy()) * 255
# cmap = cmap.astype('uint8')
# obj.module_manager.scalar_lut_manager.lut.table = cmap
# # mlab.colorbar(object=obj, title=r'$w$', orientation='vertical', nb_labels=3)
# mlab.draw()

# # =============================

# Q_min = -0.11
# Q_max = -0.1
# # We create a scalar field with Q as the scalar
# src = mlab.pipeline.scalar_field(Q)

# # Q-criterion
# volume = mlab.pipeline.volume(src, vmin=Q_min, vmax=Q_max, color=(0,1,1))

# # Retrieve the LUT of the surf object.
# lut = volume.module_manager.scalar_lut_manager.lut.table.to_array()

# # The lut is a 255x4 array, with the columns representing RGBA
# # (red, green, blue, alpha) coded with integers going from 0 to 255.

# # We modify the alpha channel to add a transparency gradient
# lut[:, -1] = np.linspace(0, 255, 256)
# # and finally we put this LUT back in the surface object. We could have
# # added any 255*4 array rather than modifying an existing LUT.
# volume.module_manager.scalar_lut_manager.lut.table = lut

# # adjust camera
# mlab.view(azimuth=240, elevation=70, distance=0.9*len(x), focalpoint=[0.5*len(x), 0.5*len(y), 0.2*len(z)], roll=None, reset_roll=True, figure=fig)

# # # save figure
# # filename = f'windmill_slice-w_volume-Q_t{t_smpl:08d}.png'
# # mlab.savefig(filename, size=(50,50)) #, figure=None)#, magnification='auto') # change size for adjusting resolution

# mlab.show()

### 3x3 turbine

In [13]:
bgcolor = (1,1,1) # background color
fig = mlab.figure(fgcolor=(1, 1, 1), bgcolor=bgcolor, size=(3072, 2304))

# plot bottom slice
slc = 30  # slice number of the actual slice to be plotted
# duplicate value of slice to all the other slices
vector = u.copy()
vector[:,:,:] = vector[:,:,slc].reshape( (len(x), len(y), 1) )
# plot slice but the bottom (0th) slice. It'll have the value of slice `slc`
mag = np.abs(vector).max()
obj = mlab.volume_slice(vector, plane_orientation='z_axes', slice_index=0, line_width=0, colormap='bwr', vmin=-5, vmax=15)
# matplotlib colormap
values = np.linspace(0., 1., 256)
cmap = cm.get_cmap('Greys_r')(values.copy()) * 255
cmap = cmap.astype('uint8')
obj.module_manager.scalar_lut_manager.lut.table = cmap
# mlab.colorbar(object=obj, title=r'$w$', orientation='vertical', nb_labels=3)
mlab.draw()

# =============================

# Q_min = -0.11
# Q_max = -0.1
Q_min = -0.441 # 0.47 0.45
Q_max = -0.44
# We create a scalar field with Q as the scalar
src = mlab.pipeline.scalar_field(Q)

# Q-criterion
volume = mlab.pipeline.volume(src, vmin=Q_min, vmax=Q_max, color=(1,0,0))

# Retrieve the LUT of the surf object.
lut = volume.module_manager.scalar_lut_manager.lut.table.to_array()

# The lut is a 255x4 array, with the columns representing RGBA
# (red, green, blue, alpha) coded with integers going from 0 to 255.

# We modify the alpha channel to add a transparency gradient
lut[:, -1] = np.linspace(0, 255, 256)
# and finally we put this LUT back in the surface object. We could have
# added any 255*4 array rather than modifying an existing LUT.
volume.module_manager.scalar_lut_manager.lut.table = lut

# adjust camera
# Inset - close
# mlab.view(azimuth=270, elevation=10, distance=0.5*len(x), focalpoint=[0.5*len(x), 0.5*len(y), 0.2*len(z)], roll=None, reset_roll=True, figure=fig)
# Inset - away
# mlab.view(azimuth=270, elevation=70, distance=0.3*len(x), focalpoint=[0.55*len(x), 0.2*len(y), 0.2*len(z)], roll=None, reset_roll=True, figure=fig)
# Full image
mlab.view(azimuth=270, elevation=70, distance=1.2*len(x), focalpoint=[0.5*len(x), 0.4*len(y), 0.2*len(z)], roll=None, reset_roll=True, figure=fig)

# # save figure
# filename = f'windmill_slice-u_volume-Q_t{t_smpl:08d}_inset3.png'
# mlab.savefig(filename, size=(50,50)) #, figure=None)#, magnification='auto') # change size for adjusting resolution

mlab.show()

/var/folders/30/4m4szwf162x06l9rfddfgp90c0khl7/T/ipykernel_1718/37388441.py:14: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('Greys_r')(values.copy()) * 255


### Isosurface colored with another scalar (not volume rendering; low quality isosurfaces)

In [14]:
# bgcolor = (0,0,0) # background color
# fig = mlab.figure(fgcolor=(1, 1, 1), bgcolor=bgcolor, size=(3072, 2304))

# # plot bottom slice
# slc = 50  # slice number of the actual slice to be plotted
# # duplicate value of slice to all the other slices
# vector = w.copy()
# vector[:,:,:] = vector[:,:,slc].reshape( (len(x), len(y), 1) )
# # plot slice but the bottom (0th) slice. It'll have the value of slice `slc`
# mag = np.abs(vector).max()
# obj = mlab.volume_slice(vector, plane_orientation='z_axes', slice_index=0, line_width=0, colormap='bwr', vmin=-mag, vmax=mag)
# # matplotlib colormap
# values = np.linspace(0., 1., 256)
# cmap = cm.get_cmap('BlueBlackRed')(values.copy()) * 255
# cmap = cmap.astype('uint8')
# obj.module_manager.scalar_lut_manager.lut.table = cmap
# # mlab.colorbar(object=obj, title=r'$w$', orientation='vertical', nb_labels=3)
# mlab.draw()

# # =============================

# Q_min = -0.34
# Q_max = -0.33
# # We create a scalar field with Q as the scalar
# src = mlab.pipeline.scalar_field(Q)

# # And we add the TKE as an additional array
# # This is a tricky part: the layout of the new array needs to be the same
# # as the existing dataset, and no checks are performed. The shape needs
# # to be the same, and so should the data. Failure to do so can result in
# # segfaults.
# src.image_data.point_data.add_array(Umag.ravel())
# # We need to give a name to our new dataset.
# src.image_data.point_data.get_array(1).name = 'TKE'
# # Make sure that the dataset is up to date with the different arrays:
# src.update()

# # We select the 'scalar' attribute, ie Q
# src2 = mlab.pipeline.set_active_attribute(src,
#                                     point_scalars='scalar')

# # # Cut isosurfaces of Q
# # contour = mlab.pipeline.contour(src2)
# # Cut isosurfaces of the norm
# contour = mlab.pipeline.contour(src2)
# min_c = Q_min #min(contour.filter._data_min * 1.05,contour.filter._data_max)
# max_c = Q_max #max(contour.filter._data_max * 0.95,contour.filter._data_min)
# # plotIsoSurfaceContours = [ max(min(max_c,x),min_c) for x in plotIsoSurfaceContours ]
# # contour.filter.contours = plotIsoSurfaceContours
# contour.filter.contours = np.linspace(Q_min, Q_max, 10).tolist()

# # Now we select the 'TKE' attribute, ie the TKE
# contour2 = mlab.pipeline.set_active_attribute(contour,
#                                     point_scalars='TKE')

# # And we display the surface. The colormap is the current attribute: TKE.
# iso = mlab.pipeline.surface(contour2, colormap='jet')
# # mlab.colorbar(object=iso, title=r'TKE', orientation='vertical', nb_labels=3)

# # adjust camera
# mlab.view(azimuth=-120, elevation=70, distance=1.2*len(x), focalpoint=[0.5*len(x), 0.6*len(y), 0.5*len(z)], roll=None, reset_roll=True, figure=fig)

# mlab.show()